# Analysis of proposed Amtrak station closures and voting preferences

I was curious about the relationship between Amtrak closures and 2016 voter patterns. I suspected that it was going to be 90% places that voted for Trump. It turns out I was a bit high. About 70% of closure cities voted for Trump. Amtrak city closures will take away one of the more affordable and pleasureable modes of travel for many people across the country and travelers from other countries as well.

### Data sources
1. Amtrak station closure list from [Mass Transit Magazine](http://www.masstransitmag.com/press_release/12322126/220-cities-losing-all-passenger-train-service-per-trump-elimination-of-all-federal-funding-for-amtraks-national-network-trains)
2. Election results from [Tony McGovern](https://github.com/tonmcg/County_Level_Election_Results_12-16)
3. Geocoding of cities with google geocoding API
4. Charting via Plotly

In [ ]:
import pandas
import requests
import os
import urllib
import re
import json
import time
import plotly.offline as plt
import plotly.graph_objs as plt_graph

plt.init_notebook_mode(connected=True)

In [ ]:
# for caching of geocode results from google (or any general URL)
def get_url_with_cache(url):
    pts = urllib.parse.urlparse(url)
    dir_name = "/Users/tommy/webdata/" + pts.netloc
    file_name = urllib.parse.unquote(pts.path + "_" +  pts.params + "_" +  re.sub("key=[^&]+", "", pts.query))
    file_name = re.sub("[^a-zA-Z0-9]+", "_", file_name) + ".html"
    if not os.path.exists(dir_name):
        os.mkdir(dir_name)
    if not os.path.exists(dir_name + "/" + file_name):
        resp = requests.get(url)
        fw = open(dir_name + "/" + file_name, "wb")
        fw.write(resp.content)
        fw.close()
    fr = open(dir_name + "/" + file_name, "r")
    contents = fr.read()
    fr.close()
    return contents

In [ ]:
stations_df = pandas.read_csv("amtrak-station-closure-list.txt")
election_results_url = "https://raw.githubusercontent.com/tonmcg/" + \
                        "County_Level_Election_Results_12-16/master/" + \
                        "2016_US_County_Level_Presidential_Results.csv"
county_results_df = pandas.read_csv(election_results_url)

In [ ]:
base_geocode_url = "https://maps.googleapis.com/maps/api/geocode/json?address=%(address)s&key=%(api_key)s"
counties = []
for row_num, d in stations_df.iterrows():
    address = urllib.parse.quote("%s, %s" % (d["city"], d["state"]))
    url = base_geocode_url % {"address":address, "api_key": os.environ["GOOGLE_API_KEY"]}
    js_txt = get_url_with_cache(url)
    js = json.loads(js_txt)
    county_name = '!Unknown'
    for loc in js['results'][0]['address_components']:
        if 'administrative_area_level_2' in loc['types']:
            county_name = loc['short_name']
    counties.append(county_name)
stations_df['county'] = counties


In [ ]:
county_results_df.rename(columns={'state_abbr': 'state', 'county_name':'county'}, inplace=True)
county_results_df.set_index(['state', 'county'], inplace=True)

In [ ]:
final_results = []

for row_num, d in stations_df.iterrows():
    try:
        cr = county_results_df.ix[d['state']].ix[d['county']]
        final_results.append([d['city'] + ', ' + d['state'], d['county'], cr['per_dem'], cr['per_gop']])
    except:
        pass

final_df = pandas.DataFrame(final_results, columns=['city', 'county', 'per_dem', 'per_gop'])
final_df['norm_dem'] = final_df['per_dem'] / (final_df['per_dem'] + final_df['per_gop'])
final_df = final_df.sort_values(['norm_dem'], ascending=[False])

In [ ]:
dems = plt_graph.Bar(
    y=final_df['city'],
    x=final_df['per_dem'] / (final_df['per_dem'] + final_df['per_gop']),
    name='Democrat Percent',
    orientation = 'h',
    marker = dict(
        color = 'rgba(30, 40, 200, 0.9)'
    )
)

reps = plt_graph.Bar(
    y=final_df['city'],
    x=final_df['per_gop'] / (final_df['per_dem'] + final_df['per_gop']),
    name='Republican Percent',
    orientation = 'h',
    marker = dict(
        color = 'rgba(200, 40, 30, 0.9)'
    )
)

layout = plt_graph.Layout(
    barmode='stack',
    height=800,
    margin=plt_graph.Margin(
        l=150
    )
)

fig = plt_graph.Figure(data=[dems, reps], layout=layout)

plt.iplot(fig)
